In [24]:
import warnings

import xarray
from zarr.errors import UnstableSpecificationWarning

In [25]:
warnings.filterwarnings(
    "ignore",
    message="Consolidated metadata is currently not part in the Zarr format*",
    category=UserWarning,
)
warnings.filterwarnings(
    "ignore",
    message="The data type*",
    category=UnstableSpecificationWarning,
)

In [26]:
# too much baggage
#source_store["/aqueduct/depth_fluvial"].chunk(chunkspec).to_zarr("../raster/target.zarr", group="/aqueduct/depth_fluvial", mode="w", align_chunks=True)

In [27]:
source = "../raster/stack.v3.zarr"
target = "../raster/target.zarr"
ll_size = 1024

store = xarray.open_groups(source)
groups = [k for k in store.keys() if k.count('/') == 2]

In [ ]:
for group in reversed(groups):
    print(group)
    chunkspec = dict(store[group].sizes)
    chunkspec["lat"] = ll_size
    chunkspec["lon"] = ll_size
    xarray.open_zarr(source, group=group).chunk(chunkspec).to_zarr(target, group=group, mode="w", align_chunks=True)

In [ ]:
# Read a large-ish chunk based on row and column index slices using `isel`
# ds = store["/aqueduct/depth_fluvial"].isel(lon=slice(0,1024), lat=slice(0,1024)).compute()
# ds

In [ ]:
ds = store["/aqueduct/depth_fluvial"].sel(rp=1000, epoch="2080") # select without materialising
ds.sel(lon=-138995, lat=6753615, method="nearest").compute()
# use method=nearest for lat/lon to find the nearest grid cell with any point-click coordinates
# !! NB actually in EPSG:3857 for consistency with TIFF grids. Should be renamed in the data to x/y (lat/lon is misleading)

<xarray.Dataset> Size: 660B
Dimensions:        (hazard: 1, rcp: 3, gcm: 6)
Coordinates:
  * hazard         (hazard) <U7 28B 'fluvial'
  * rcp            (rcp) <U8 96B 'baseline' '4.5' '8.5'
  * gcm            (gcm) <U14 336B 'WATCH' 'NorESM1-M' ... 'MIROC-ESM-CHEM'
    epoch          <U8 32B '2080'
    lat            float64 8B 6.754e+06
    lon            float64 8B -1.39e+05
    rp             int64 8B 1000
Data variables:
    depth_fluvial  (hazard, rcp, gcm) float64 144B nan nan nan ... 0.4647 0.4791